In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import tensorflow as tf
import time

In [5]:
def tf_rad2deg(rad):
    pi_on_180 = 0.017453292519943295
    return rad / pi_on_180

def dihedral(p):
    """Praxeolitic formula
    1 sqrt, 1 cross product"""
    p0 = p[0]
    p1 = p[1]
    p2 = p[2]
    p3 = p[3]

    b0 = -1.0*(p1 - p0)
    b1 = p2 - p1
    b2 = p3 - p2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    b1 /= np.linalg.norm(b1)

    # vector rejections
    # v = projection of b0 onto plane perpendicular to b1
    #   = b0 minus component that aligns with b1
    # w = projection of b2 onto plane perpendicular to b1
    #   = b2 minus component that aligns with b1
    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

# takes 1 dimensional tensor and outputs an angle
def dihedral_tf1(p):
    p0 = tf.gather(p, 0)
    p1 = tf.gather(p, 1)
    p2 = tf.gather(p, 2)
    p3 = tf.gather(p, 3)
    
    b0 = -1.0 * (tf.subtract(p1, p0))
    b1 = tf.subtract(p2, p1)
    b2 = tf.subtract(p3, p2)
    
    b1 = tf.divide(b1, tf.norm(b1))
    
    v = tf.subtract(b0, tf.multiply(tf.tensordot(b0, b1, 1), b1))
    w = tf.subtract(b2, tf.multiply(tf.tensordot(b2, b1, 1), b1))
    
    x = tf.tensordot(v, w, 1)
    y = tf.tensordot(tf.cross(b1, v), w, 1)
    
    return tf_rad2deg(tf.atan2(y,x))

# takes 2 dimensional tensor (K, 4) and outputs K angles
def dihedral_tf2(p):
    p0 = tf.gather(p, 0, axis=1)
    p1 = tf.gather(p, 1, axis=1)
    p2 = tf.gather(p, 2, axis=1)
    p3 = tf.gather(p, 3, axis=1)
    
    b0 = -1.0 * (tf.subtract(p1, p0))
    b1 = tf.subtract(p2, p1)
    b2 = tf.subtract(p3, p2)
    
    b1 = tf.divide(b1, tf.norm(b1, axis=1, keepdims=True))
    
    v = tf.subtract(b0, tf.einsum('b,bi->bi', tf.einsum('bi,bi->b', b0, b1), b1))
    w = tf.subtract(b2, tf.einsum('b,bi->bi', tf.einsum('bi,bi->b', b2, b1), b1))
    
    x = tf.reduce_sum( tf.multiply( v, w ), 1, keepdims=True )
    y = tf.reduce_sum( tf.multiply( tf.cross(b1, v), w ), 1, keepdims=True )

    return tf_rad2deg(tf.atan2(y,x))

# takes a 3 dimensional tensor (N, K, 4) and outputs (N,K) angles
def dihedral_tf3(p):
    p0 = tf.gather(p, 0, axis=2)
    p1 = tf.gather(p, 1, axis=2)
    p2 = tf.gather(p, 2, axis=2)
    p3 = tf.gather(p, 3, axis=2)
    
    b0 = -1.0 * (tf.subtract(p1, p0))
    b1 = tf.subtract(p2, p1)
    b2 = tf.subtract(p3, p2)
    
    b1 = tf.divide(b1, tf.norm(b1, axis=2, keepdims=True))
    b1 = tf.where(tf.is_nan(b1), tf.ones_like(b1), b1)
    
    v = tf.subtract(b0, tf.einsum('bi,bij->bij', tf.einsum('bij,bij->bi', b0, b1), b1))
    w = tf.subtract(b2, tf.einsum('bi,bij->bij', tf.einsum('bij,bij->bi', b2, b1), b1))
    
    x = tf.reduce_sum( tf.multiply( v, w ), 2, keepdims=True )
    y = tf.reduce_sum( tf.multiply( tf.cross(b1, v), w ), 2, keepdims=True )

    return tf_rad2deg(tf.atan2(y,x))

In [6]:
p0 = np.array([24.969, 13.428, 30.692]) # N
p1 = np.array([24.044, 12.661, 29.808]) # CA
p2 = np.array([22.785, 13.482, 29.543]) # C
p3 = np.array([21.951, 13.670, 30.431]) # O
# result: -71

p_org = np.array([[
                [ 1,           0,         0     ],
                [ 0,           0,         0     ],
                [ 0,           0,         1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ]
            ],[
                [ 1,           0,         0     ],
                [ 0,           0,         0     ],
                [ 0,           0,         1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ]
            ],[
                p0,
                p1,
                p2,
                p3,
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ]
            ]])
print(p_org.shape)

p = tf.convert_to_tensor(p_org[2])
# p1 = np.expand_dims(p1, axis=0)
p = p[None,:,:,None]
p = tf.extract_image_patches(p,
  ksizes=[1, 4, 3, 1],
  strides=[1, 1, 1, 1],
  rates=[1, 1, 1, 1],
  padding='VALID')
p = tf.reshape(tf.squeeze(p), [-1, 4, 3])
angles = dihedral_tf1(tf.convert_to_tensor(p_org[2, 0:4]))
angles2 = dihedral_tf2(p)

p2 = tf.convert_to_tensor(p_org)
# p1 = np.expand_dims(p1, axis=0)
p2 = p2[:,:,:,None]
p2 = tf.extract_image_patches(p2,
  ksizes=[1, 4, 3, 1],
  strides=[1, 1, 1, 1],
  rates=[1, 1, 1, 1],
  padding='VALID')
p2 = tf.reshape(tf.squeeze(p2), [3, -1, 4, 3])
angles3 = dihedral_tf3(p2)

with tf.Session() as sess:
    angles_, angles2_, angles3_, pshape, p2shape = sess.run([angles, angles2, angles3, tf.shape(p), tf.shape(p2)])
    
np.array(angles_), np.array(angles2_), np.array(angles3_[2])

(3, 7, 3)


(array(-71.21515115), array([[ -71.21515115],
        [-127.82961937],
        [   0.        ],
        [          nan]]), array([[ -71.21515115],
        [-127.82961937],
        [   0.        ],
        [   0.        ]]))